In [13]:
import pickle
import torch
from torch import nn
import torchvision
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm

In [14]:
from modules.utils import *

In [15]:
ensembl_trx = pickle.load(open('data/ensembl_trx.pkl', 'rb'))
trx_orfs = pickle.load(open('data/trx_orfs.pkl', 'rb'))
dataset = pickle.load(open('data/dataset.pkl', 'rb'))
alt_dataset = pickle.load(open('data/alt_dataset.pkl', 'rb'))

In [16]:
pc_dataset = {x:y["sequence"] for (x,y) in ensembl_trx.items() if x in dataset}
pseudo_dataset = {x:y["sequence"] for (x,y) in ensembl_trx.items() if x in alt_dataset and ensembl_trx[x]["biotype"] == "pseudogene"}

In [17]:
clustering_dataset = pc_dataset | pseudo_dataset

In [18]:
data = [(x,y) for x,y in clustering_dataset.items()]

In [19]:
def build_fasta(data, filename):
    fasta_sequences = []
    for i, trx in enumerate(data):
        trx_id, seq = trx[0], trx[1]
        header = f'>{trx_id}'
        fasta_sequence = header + '\n' + seq.upper()
        fasta_sequences.append(fasta_sequence)
    
    fasta_text = '\n'.join(fasta_sequences)
    
    with open(filename + '.fasta', 'w') as fasta_file:
        fasta_file.write(fasta_text)

In [20]:
build_fasta(data, 'mydb')

In [21]:
len(data)

36339

In [22]:
import csv

In [29]:
results = dict()
with open('data/results.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for n, row in enumerate(reader):
        trx = row[0]
        match = row[1]
        if trx == match:
            continue
        if trx not in results:
            results[trx] = [match]
        else:
            results[trx].append(match)

In [30]:
len(results)

15523

In [8]:
import pandas as pd
from tqdm.notebook import tqdm

In [9]:
cols = rows = [x for x in dataset.keys()]

In [10]:
df = pd.DataFrame(0, columns=cols, index=rows)

In [11]:
for trx, matches in tqdm(results.items()):
    for match, bit in matches:
        df.loc[trx, match] = float(bit)

  0%|          | 0/8035 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
psicube_trx = dict()
with open('psicube_trx.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for n, row in enumerate(reader):
        if n == 0:
            cols = row
            continue
        line = dict(zip(cols, row))
        pseudo_trx = line["ID"].split(".")[0]
        if "Parent transcript" not in line:
            continue
        parent_trx = line["Parent transcript"].split(".")[0]
        psicube_trx[pseudo_trx] = parent_trx

In [37]:
psicube_trx = {x:y for (x,y) in psicube_trx.items() if x in ensembl_trx and y in ensembl_trx}

In [42]:
count = 0
notin = 0
for trx, parent in psicube_trx.items():
    if trx not in results:
        notin += 1
        continue
    if parent in results[trx]:
        count += 1